<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2023notebooks/2023_1027Knd_Ijn_Ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%config InlineBackend.figure_format = 'retina'
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

* date: 2023_1110
* author: 浅川伸一
* filename: 2023_1027Knd_Ijn_Ask.ipynb 名前変更，旧名は `2023_1027Knd_Ijn_Ask_s2p_p2s.ipynb` 理由は orthgraphy も追加したら。

# 符号化器‐復号化器 (encoder-decoder a.k.a seq2seq) モデルによる，単語認識過程 beyond triangle

`fit_seq2seq()`, `eval_seq2seq()` は，encoder 側が 系列データでも，埋め込みベクトルでも動作する。
従って，o2o, o2p, p2o, p2p, s2o, s2p の 6 モデルはこれでよいようだ。
残された，o2s, p2s, s2s を開発すれば良い。

<center>
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2006Kello_fig4.svg" style="width:39%">
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2006Kello_junction_fig5.svg" style="width:39%">
<!-- <img src="2006Kello_fig4.svg" width="39%"><img src="2006Kello_junction_fig5.svg" style="width:39%"> -->
<div style="background-color:lavender;text-align:left;width:66%">
左: RNN を用いた符号化器 (encoder) -復号化器 (decoder) モデル。
右: Kello らの結節点 (junction) モデル。中央の語彙ノード (lexical nodes) 上の数字 45263 は，交差点モデルが扱うことが可能な語彙数。<br/>
左: Kello+2006 Fig.4, 右: Kello+2006 Fig5
</div>
</center>


<center>
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2004Harm_Seidenberg_fig1_extended.svg" style="width:39%">
<!-- <img src="2004Harm_Seidenberg_fig1_extended.svg" width="39%"> -->
<div style="background-color:lavender;text-align:center;width:44%">
Harm\&Seidenberg2004 Fig. 1 を改変。
</div>
</center>

1. O(rthgraphy), P(honology), S(emnatics) のそれぞれに対して，ソースとターゲットと見立てた，9 つのデータセット，モデルを用意した。
モデル名を下表に示す。
表中の x2y は，ソースが x [o,p,s] でターゲットが y [o,p,s] であるモデルを意味する。
カッコ内は，ソースとターゲットのそれぞれが，系列データであれば Seq であり，埋め込みベクトルデータであれば Vec である。

|source\target   | O   | P   |  S |
|:--:|:--:|:---:|:--:|
| O | o2o (Seq2seq)| o2p (Seq2Seq)| o2s (Seq2Vec)|
| P | p2o (Seq2Seq)| p2p (Seq2Seq)| p2s (Seq2Vec)|
| S | s2o (Vec2Seq)| s2p (Vec2Seq)| s2s (Vec2Vec)|

ソースからターゲットへと系列データかベクトル埋め込みデータかによって，モデルは 4 種類に分類できる。

1. 系列から系列へ: 4 (o2o, o2p, p2o, p2p)，
2. 系列からベクトル埋め込みへ: 2 (o2s, p2s)
3. ベクトル埋め込みから系列へ: 2 (s2o, s2p)
4. ベクトル埋め込みからベクトル埋め込み 1

## メモ

1. 2023_1027 に近藤先生には，このコードのプロトタイプをお見せした。
すなわち GitHub にアップロード済である。
このファイルには，その後の改良が加えられている。
ただし，最初のセルで MeCab をソースコードからダウンロードして，コンパイル & インストールに時間を要したため，実施まではお見せしていない。

2. MeCab の使用は，未知語が入力として与えられた場合，仮のヨミを得るために使用している。
上記の役割を除けば MeCab は不要だと判断し，MeCab の使用を中止した


<center>
<!-- <img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2004Harm_Seidenberg_fig4c.svg"><br/>
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2004Harm_Seidenberg_fig4d.svg"><br/> -->
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2004Harm_Seidenberg_fig4ab.svg"><br/>
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2004Harm_Seidenberg_fig4c.svg">
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2004Harm_Seidenberg_fig4d.svg"><br/>
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/1999Levelt_blueprint.jpg" width=49%"><br/>
<!-- <img src="2004Harm_Seidenberg_fig4ab.svg"><br/>
<img src="2004Harm_Seidenberg_fig4c.svg">
<img src="2004Harm_Seidenberg_fig4d.svg"><br/> -->
`Harm & Seidenberg (2004)`, Figure 4 c, and d,
`Levelt 1999
</center>

<center>
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2015Loung_fig1.svg" width="24%"><br/>
ニューラル翻訳モデル。
青色がソース言語モデル，赤がターゲット言語モデルである。
ソース言語モデルの，最終時刻の中間層状態を，ターゲット言語モデルの開始時の中間層状態として用いる。
Loung+2015 Fig.1 より。    
</center>


* 文献
    * Harm & Seidenberg (2004) Computing the Meanings of Words in Reading: Cooperative Division of Labor Between Visual and Phonological Processes, Psychological Review, DOI:10.1037/0033-295X.111.3.662
    * Seq2seq 翻訳モデル: Sutskever+ (2014) Sequence to Sequence Learning with Neural Networks, [arXiv:1409.3215](https://arxiv.org/abs/1409.3215)
    * 注意つき符号化器‐復号化器モデル: Bahdanau, Cho, & Bengio (2015) NEURAL MACHINE TRANSLATION BY JOINTLY LEARNING TO ALIGN AND TRANSLATE, [arXiv:1409.0473](https://arxiv.org/abs/1409.0473)
    * もう一つの注意つき符号化器‐復号化器モデル Luong+ (2015) Effective Approaches to Attention-based Neural Machine Translation, [arXiv:1508.04025](https://arxiv.org/abs/1508.04025)


# 0 準備


## 0.1 必要なライブラリの輸入

In [ ]:
import sys
import os
import numpy as np
from tqdm.notebook import tqdm
import time
import datetime
import matplotlib.pyplot as plt

try:
    import jaconv
except ImportError:
    !pip install jaconv

try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

if isColab:
    !pip install --upgrade termcolor==1.1
from termcolor import colored

try:
    import RAM
except ImportError:
    !git clone https://github.com/ShinAsakawa/RAM.git
    import RAM

# 近藤先生との議論から音韻情報の代替案として，ローマ字表記を採用することとした。
# このとき，訓令式の表記にすることとした。ヘボン式，パスポート式ではないことに注意
try:
    from kunrei import kunrei
except ImportError:
    !wget https://shinasakawa.github.io/2023notebooks/kunrei.py -O kunrei.py
    from kunrei import kunrei

## 0.2 意味表現として word2vec による意味埋め込みベクトルを使う

In [ ]:
%%time
# word2vec のため gensim を使う
import requests
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import os
HOME = os.environ['HOME']

w2v_2017 = {
    'cbow200': 'http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz',
    'sgns200': 'http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz',
    'cbow300': 'http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid300_win20_neg20_sgns.bin.gz',
    'sgns300': 'http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
}

w2v_2021 = {
    'cbow128': { 'id': '1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb',
                'outfile': '2021_05jawiki_hid128_win10_neg10_cbow.bin.gz'},
    'sgns128': { 'id': '1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M',
                'outfile': '2021_05jawiki_hid128_win10_neg10_sgns.bin.gz'},
    'cbow200': { 'id': '1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s',
                'outfile': '2021_05jawiki_hid200_win20_neg20_sgns.bin.gz'}
}

is2017=True

if isColab:
    from google_drive_downloader import GoogleDriveDownloader as gdd

    if is2017:
        response = requests.get(w2v_2017['cbow200'])
        fname = w2v_2017['cbow200'].split('/')[-1]
        with open(fname, 'wb') as f:
            f.write(response.content)
    else:
        #訓練済 word2vec ファイルの取得
        (f_id, outfile) = w2v_2021['sgns128']['id'], w2v_2021['sgns128']['outfile']
        gdd.download_file_from_google_drive(file_id=f_id,
                                            dest_path=outfile,
                                            unzip=False,
                                            showsize=True)

if is2017:
    w2v_base = os.path.join(HOME, 'study/2016wikipedia/') if not isColab else '.'
    w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
    w2v_file = os.path.join(w2v_base, w2v_file)
else:
    w2v_base = os.path.join(HOME, 'study/2019attardi_wikiextractor.git/wiki_texts/AA') if isMac else '.'
    w2v_file = '2021_05jawiki_hid128_win10_neg10_sgns.bin'

w2v = KeyedVectors.load_word2vec_format(
    w2v_file,
    encoding='utf-8',
    unicode_errors='replace',
    binary=True)

## 0.3 データセット Psylex71_Dataset の読み込み

In [ ]:
# データセットとしての Psylex71_Dataset の読み込み
from RAM import Psylex71_Dataset

psylex71_ds = Psylex71_Dataset(max_words=30000)
print(f'psylex71_ds の単語数:{psylex71_ds.__len__()}')

### データセットのヒストグラム描画

In [ ]:
from RAM import draw_word_char_histgram
draw_word_char_histgram(_dict=psylex71_ds.data_dict, key='phon', title='音韻', figsize2=(8,3))

## 0.4 psylex71_ds に存在する全単語を word2vec の埋め込みベクトル行列にする

In [ ]:
# psylex71_ds データから word2vec の埋め込みベクトル行列を得る
_words = [dct['orth'] for dct in psylex71_ds.data_dict.values()]

# gensim() の `vectors_for_all()` 関数を持ちて，望む語彙で構成される word2vec 単語埋め込みモデルを作成
w2v_psylex71 = w2v.vectors_for_all(_words)

# NaN データが入っている可能性がるので変換
w2v_psylex71.vectors = np.nan_to_num(w2v_psylex71.vectors)
print(f'w2v_psylex71.vectors.shape:{w2v_psylex71.vectors.shape}')
words = w2v_psylex71.index_to_key
#len(words)

## 0.5 psylex71 データセット中の単語における w2v の表示テスト

In [ ]:
Wrd = input('単語を入力してください:')
color = 'blue'
while (Wrd != ""):
    if Wrd in w2v_psylex71:
        Idx = w2v_psylex71.key_to_index[Wrd]
        print(f'入力単語 Wrd:{colored(Wrd, color, attrs=["bold"])},',
              f'対応する単語番号 Idx:{colored(Idx, color, attrs=["bold"])},',
              f'w2v_psylex71.get_index({Wrd}):{colored(w2v_psylex71.get_index(Wrd), color, attrs=["bold"])}')
    else:
        print(colored(f'{Wrd} という単語はありません。','red', attrs=['bold']))
    Wrd = input('単語を入力してください (終了するには改行のみを入力):')

## 0.6 書記素リストの作成

In [ ]:
import RAM

def _grapheme(words=words):
    """必要と思われる書記素リストを返す"""

    num_alpha='０１２３４５６７８９ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ'
    hira = 'あいうえおかがきぎくぐけげこごさざしじすずせぜそぞただちぢつづてでとどなにぬねのはばぱひびぴふぶぷへべぺほぼぽまみむめもやゆよらりるれろわゐゑをんぁぃぅぇっゃゅょゎ'+'ゔ'
    kata = 'アイウエオカガキギクグケゲコゴサザシジスズセゼソゾタダチヂツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポマミムメモヤユヨラリルレロワヰヱヲン'+'ヴヷヸヹヺァィゥヵヶェォッャョュヮ'
    symbols='、。，．・：；？！゛゜´｀¨＾‾＿ヽヾゝゞ〃仝々〆〇ー—‐／＼〜‖｜…‥‘’“”（）〔〕［］｛｝〈〉《》「」『』【】＋−±×÷＝≠＜＞≦≧∞∴♂♀°′″℃¥＄¢£％＃＆＊＠§☆★○●◎◇' + '◆□■△▲▽▼※〒→←↑↓〓∈∋⊆⊇⊂⊃∪∩∧∨¬⇒⇔∀∃∠⊥⌒∂∇≡≒≪≫√∽∝∵∫∬Å‰♯♭♪†‡¶◯'
    #greek='ΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩαβγδεζηθικλμνξοπρστυφχψω'
    #rosian='АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    #digit_symbols='①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳⑴⑵⑶⑷⑸⑹⑺⑻⑼⑽⑾⑿⒀⒁⒂⒃⒄⒅⒆⒇❶❷❸❹❺❻❼❽❾⒈⒉⒊⒋⒌⒍⒎⒏⒐'
    #alpha_symbols='ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩⅪⅫⅰⅱⅲⅳⅴⅵⅶⅷⅸⅹⅺⅻ⒜⒝⒞⒟⒠⒡⒢⒣⒤⒥⒦⒧⒨⒩⒪⒫⒬⒭⒮⒯⒰⒱⒲⒳⒴⒵'
    #units='㎜㎟㎝㎠㎤㎡㎥㎞㎢㎎㎏㏄㎖㎗ℓ㎘㎳㎲㎱㎰℉㏔㏋㎐㎅㎆㎇№㏍℡'
    #suits='♤♧♡♢♠♣♥♦〠☎〄☞☜☝☟⇆⇄⇅⇨⇦⇧⇩'
    #etc='①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ㍉㌔㌢㍍㌘㌧㌃㌶㍑㍗㌍㌦㌣㌫㍊㌻㎜㎝㎞㎎㎏㏄㎡㍻〝〟№㏍℡㊤㊥㊦㊧㊨㈱㈲㈹㍾㍽㍼≒≡∫∮∑√⊥∠∟⊿∵∩∪㊙'
    #etc2='㍉㌢㍍㌔㌖㌅㌳㍎㌃㌶㌘㌕㌧㍑㍊㌹㍗㌍㍂㌣㌦㌻㌫㌀㌞㌪㌱㍇㍾㍽㍼㍻㍿∮∟⊿〝'

    # RAM で作成済の常用漢字リストを用いて単漢字リストを作成
    # 平成 22 年の改定により常用漢字は 2136 文字ある
    chars_list = [ch for ch in num_alpha+hira+kata+symbols]+ RAM.chars_joyo().char_list
    not_chars_list = []
    for wrd in tqdm(words):
        for ch in wrd:
            if (ch not in chars_list) and (ch not in not_chars_list):
                not_chars_list.append(ch)
    not_chars_list = sorted(not_chars_list)
    grapheme = chars_list + not_chars_list
    # 上記の処理により grapheme には 2768 文字である。
    # これに特殊トークン 4 つ ['<PAD>', '<SOW>', '<EOW>', '<UNK>'] を加えたリストを返す

    return ['<PAD>', '<SOW>', '<EOW>', '<UNK>'] + grapheme

grapheme = _grapheme()

## 0.7 データセット定義

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import gensim

class psylex71_w2v_Dataset(Dataset):
    def __init__(self,
                 #direction='s2p',  # ['s2p', 'p2s']
                 source='seme',    # エンコーダ用 入力データ, ['orth', seme', 'phon'] のいずれか一つ
                 target='phon',    # デコーダ用 出力データ ,  ['orth', seme', 'phon'] のいずれか一つ
                 w2v:gensim.models.keyedvectors.KeyedVectors=w2v_psylex71,
                 old_ds:RAM.dataset.Psylex71_Dataset=psylex71_ds,
                 #mecab_yomi=yomi,
                 grapheme:list=grapheme,
                ):

        super().__init__()
        self.ds_name = 'psylex71_'+source+"2"+target
        self.source, self.target = source, target

        self.w2v = w2v
        self.old_ds = old_ds
        #self.mecab_yomi = yomi         # 未知の単語が入力された場合 MeCab を使って読みをえるため
        self.grapheme = grapheme

        self.words = w2v.index_to_key  # gensim の KeyedVectors を利用して単語リストとする
        self.W = w2v.vectors

        # 訓令式に従った日本語ローマ字表記 `kurei.py` 参照
        self.phoneme = ['<PAD>', '<SOW>', '<EOW>', '<UNK>', # 特殊トークン，純に，埋め草，語頭，語末，未知
                        'a', 'i', 'u', 'e', 'o',            # 母音
                        'a:', 'i:', 'u:', 'e:', 'o:',       # 長母音
                        'N', 'Q',                           # 撥音，拗音
                        'b', 'by', 'ch', 'd', 'dy', 'f', 'g', 'gy', 'h', 'hy', # 子音
                        'j', 'k', 'ky', 'm', 'my', 'n', 'ny',  'p', 'py', 'r', # 子音
                        'ry', 's', 'sy', 't', 'ty', 'w', 'y', 'z', 'zy']       # 子音



    def __getitem__(self, idx:int):
        wrd = self.words[idx]

        if self.source == 'phon':
            src = torch.LongTensor(self.wrd2phon_ids(wrd))
        elif self.source == 'seme':
            src = torch.tensor(self.w2v.get_vector(idx))
        elif self.source == 'orth':
            src = torch.LongTensor(self.wrd2orth_ids(wrd))
        else:
            src = None

        if self.target == 'phon':
            tgt = torch.LongTensor(self.wrd2phon_ids(wrd))
        elif self.target == 'seme':
            tgt = torch.tensor(self.w2v.get_vector(idx))
        elif self.target == 'orth':
            tgt = torch.LongTensor(self.wrd2orth_ids(wrd))
        else:
            tgt = None

        return src, tgt

#     def __getitem__saved(self,  # 旧バージョン
#                     idx:int,
#                     direction:str=None):
#         wrd = self.words[idx]
#         if direction == None:
#             direction = self.direction
#         if direction == 'p2s':
#             X = torch.LongTensor(self.wrd2phon_ids(wrd))
#             y = torch.tensor(self.w2v.get_vector(idx), dtype=torch.float16)
#         else:
#             y = torch.LongTensor(self.wrd2phon_ids(wrd))
#             X = torch.tensor(self.w2v.get_vector(idx), dtype=torch.float16)

#         return X, y


    def __len__(self):
        return len(self.w2v)

    def getitem(self,
                idx:int):
        wrd = self.words[idx]
        _yomi = self.wrd2yomi(wrd)
        _yomi = kunrei(_yomi).split(' ')
        phon_ids = [self.phoneme.index(idx) for idx in _yomi]
        orth_ids = [self.grapheme.index(idx) for idx in wrd]
        return wrd, _yomi, phon_ids, orth_ids

    def source_ids2source(self, ids:list):

        if self.source == 'phon':
            return self.phon_ids2phn(ids)
        elif self.source == 'orth':
            return self.orth_ids2orth(ids)
        elif self.source == 'seme':
            wrd = self.getitem(ids)[0]
            return w2v.similar_by_word(wrd)
        else:
            return None


    def target_ids2target(self, ids:list):

        if self.target == 'phon':
            return self.phon_ids2phn(ids)
        elif self.target == 'orth':
            return self.orth_ids2orth(ids)
        elif self.target == 'seme':
            wrd = self.getitem(ids)[0]
            return w2v.similar_by_word(wrd)
        else:
            return None


    def wrd2orth_ids(self, wrd:str)->list:
        ids = [self.grapheme.index(ch) for ch in wrd]
        ids = [self.grapheme.index('<SOW>')] + ids + [self.grapheme.index('<EOW>')]
        #ids = [[self.grapheme.index('<SOW>')] + ids + [self.grapheme.index('<EOW>')]]
        return ids

    def wrd2phon_ids(self, wrd:str)->list:
        _yomi = self.wrd2yomi(wrd)
        _yomi = kunrei(_yomi).split(' ')
        ids = [self.phoneme.index(idx) for idx in _yomi]
        ids = [self.phoneme.index('<SOW>')] + ids + [self.phoneme.index('<EOW>')]
        return ids

    def get_wrdidx_from_word(self, wrd:str):
        if wrd in self.words:
            wrd_idx = self.w2v.get_index(wrd)
        else:
            wrd_idx = -1
        return wrd_idx

    def wrd2emb(self, wrd:str)->np.ndarray:
        if wrd in self.words:
            return self.w2v.get_vector(wrd)
        else:
            return None

    def wrd2wrd_ids(self, wrd:str)->int:
        if wrd in self.words:
            return self.words.index(wrd)
        else:
            return None

    def orth_ids2orth(self,
                      ids:np.ndarray)->str:
    #def orth_ids2orth(self, ids:list)->str:
        ret = [self.grapheme[idx] for idx in ids]
        return ret

    def wrd_idx2wrd(self, idx:int)->str:
        if 0 <= idx and idx < len(self.words):
            return self.words[idx]
        else:
            return None

    def wrd2onehot(self, wrd:str)->np.ndarray:
        ret = np.zeros((self.W.shape[0],), dtype=np.int32)
        if wrd in self.words:
            ret[self.w2v.get_index(wrd)] = 1
            return ret
        else:
            return None

    def phon_ids2phn(self, ids:np.ndarray):
        ret = "".join([self.phoneme[idx] for idx in ids])
        return ret

    def wrd2yomi(self, wrd:str)->list:
        if wrd in self.words:
            _yomi = self.old_ds.orth2info_dict[wrd]['ヨミ']
        else:
            _yomi = self.mecab_yomi(wrd).strip().split()[0]
        return _yomi

    def wrd2info(self, wrd:str)->dict:
        if wrd in self.words:
            return self.old_ds.orth2info_dict[wrd]
        else:
            return None


# 全部で 9 通りのデータセットを定義
psylex71_ds_o2o = psylex71_w2v_Dataset(source='orth', target='orth')
psylex71_ds_o2p = psylex71_w2v_Dataset(source='orth', target='phon')
psylex71_ds_o2s = psylex71_w2v_Dataset(source='orth', target='seme')

psylex71_ds_p2o = psylex71_w2v_Dataset(source='phon', target='orth')
psylex71_ds_p2p = psylex71_w2v_Dataset(source='phon', target='phon')
psylex71_ds_p2s = psylex71_w2v_Dataset(source='phon', target='seme')

psylex71_ds_s2o = psylex71_w2v_Dataset(source='seme', target='orth')
psylex71_ds_s2p = psylex71_w2v_Dataset(source='seme', target='phon')
psylex71_ds_s2s = psylex71_w2v_Dataset(source='seme', target='seme')

## DataLoader の設定

# 1 モデルの定義

<div style="font-family:serif;font-size:14pt;color:purple;font-weight:900">

PyTorch RNN モデルの実装に対する注意メモ
    
* Encoder 側のデータと Decoder 側のデータそれぞれに対して Padding の処理を行う。
* Encoder 側のデータには Padding 値として `0` で埋める。
* Decoder 側のデータをモデルの forward で使う場合には、Padding 値は `0` を埋める。
* ただし，Decoder 側のデータを教師データとして使う場合には，Padding 値には -1 を用いて，埋めることに注意。
* `nn.Embedding()` のオプションに `padding_idx=O` を付け，`CrosEntropyLoss` のオプションに `ignore_index=-1` を付ける。

</div>


In [ ]:
# 全モデル共通使用するライブラリの輸入
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

## 0 共通のハイパーパラメータ宣言

In [ ]:
# Hyper parameters
n_hid = 128
n_layers = 1
bidirectional=False
batch_size = 1024

def _collate_fn(batch):
    inps, tgts = list(zip(*batch))
    inps = list(inps)
    tgts = list(tgts)
    return inps, tgts

## 1 Seq2Seq model

In [ ]:
class Seq2Seq_wAtt(nn.Module):
    """ 注意つき符号化器‐復号化器モデル
    Bahdanau, Cho, & Bengio (2015) NEURAL MACHINE TRANSLATION BY JOINTLY LEARNING TO ALIGN AND TRANSLATE, arXiv:1409.0473
    """
    def __init__(self,
                 enc_vocab_size:int,
                 dec_vocab_size:int,
                 n_hid:int,
                 n_layers:int=2,
                 bidirectional:bool=False):
        super().__init__()

        # Encoder 側の入力トークン id を多次元ベクトルに変換
        self.encoder_emb = nn.Embedding(num_embeddings=enc_vocab_size,
                                        embedding_dim=n_hid,
                                        padding_idx=0)

        # Decoder 側の入力トークン id を多次元ベクトルに変換
        self.decoder_emb = nn.Embedding(num_embeddings=dec_vocab_size,
                                        embedding_dim=n_hid,
                                        padding_idx=0)

        # Encoder LSTM 本体
        self.encoder = nn.LSTM(input_size=n_hid,
                               hidden_size=n_hid,
                               num_layers=n_layers,
                               batch_first=True,
                               bidirectional=bidirectional)

        # Decoder LSTM 本体
        self.decoder = nn.LSTM(input_size=n_hid,
                               hidden_size=n_hid,
                               num_layers=n_layers,
                               batch_first=True,
                               bidirectional=bidirectional)

        # 文脈ベクトルと出力ベクトルの合成を合成する層
        bi_fact = 2 if bidirectional else 1
        self.combine_layer = nn.Linear(bi_fact * 2 * n_hid, n_hid)

        # 最終出力層
        self.out_layer = nn.Linear(n_hid, dec_vocab_size)

    def forward(self, enc_inp, dec_inp):

        enc_emb = self.encoder_emb(enc_inp)
        enc_out, (hnx, cnx) = self.encoder(enc_emb)

        dec_emb = self.decoder_emb(dec_inp)
        dec_out, (hny, cny) = self.decoder(dec_emb,(hnx, cnx))

        # enc_out は (バッチサイズ，ソースの単語数，中間層の次元数)
        # ソース側 (enc_out) の各単語とターゲット側 (dec_out) の各単語との類似度を測定するため
        # 両テンソルの内積をとるため ソース側 (enc_out) の軸を入れ替え
        enc_outP = enc_out.permute(0,2,1)

        # sim の形状は (バッチサイズ, 中間層の次元数，ソースの単語数)
        sim = torch.bmm(dec_out, enc_outP)

        # sim の各次元のサイズを記録
        batch_size, dec_word_size, enc_word_size = sim.shape

        # sim に対して，ソフトマックスを行うため形状を変更
        simP = sim.reshape(batch_size * dec_word_size, enc_word_size)

        # simP のソフトマックスを用いて注意の重み alpha を算出
        alpha = F.softmax(simP,dim=1).reshape(batch_size, dec_word_size, enc_word_size)

        # 注意の重み alpha に encoder の出力を乗じて，文脈ベクトル c_t とする
        c_t = torch.bmm(alpha, enc_out)

        # torch.cat だから c_t と dec_out とで合成
        dec_out_ = torch.cat([c_t, dec_out], dim=2)
        dec_out_ = self.combine_layer(dec_out_)

        return self.out_layer(dec_out_)


# 以下確認作業
ds = psylex71_ds_o2p
o2p = Seq2Seq_wAtt(enc_vocab_size=len(ds.grapheme),
                   dec_vocab_size=len(ds.phoneme),
                   n_layers=n_layers,
                   bidirectional=bidirectional,
                   n_hid=n_hid).to(device)
print(o2p.eval())

## 2 Vec2Seq model

In [ ]:
class Vec2Seq(nn.Module):
    def __init__(self,
                 sem_dim:int,
                 dec_vocab_size:int,
                 n_hid:int,
                 n_layers:int=2,
                 bidirectional:bool=False):
        super().__init__()

        # 単語の意味ベクトル a.k.a 埋め込み表現 を decoder の中間層に接続するための変換層
        # 別解としては，入力層に接続する方法があるが，それはまた別実装にする
        self.enc_transform_layer = nn.Linear(
            in_features=sem_dim,
            out_features=n_hid)
        self.decoder_emb = nn.Embedding(
            num_embeddings=dec_vocab_size,
            embedding_dim=n_hid,
            padding_idx=0)

        self.decoder = nn.LSTM(
            input_size=n_hid,
            hidden_size=n_hid,
            num_layers=n_layers,
            batch_first=True,
            bidirectional=bidirectional)

        # 最終出力層
        self.bi_fact = 2 if bidirectional else 1
        self.out_layer = nn.Linear(self.bi_fact * n_hid, dec_vocab_size)

    def forward(self, enc_inp, dec_inp):
        enc_emb = self.enc_transform_layer(enc_inp)
        hnx, cnx = enc_emb.clone(), enc_emb.clone()
        hnx = hnx.unsqueeze(0)
        cnx = cnx.unsqueeze(0)

        if self.bi_fact == 2:
            hnx = hnx.repeat(2)
            cnx = cnx.repeat(2)

        dec_emb = self.decoder_emb(dec_inp)

        batch_size = enc_inp.size(0)
        exp_hid_size = self.decoder.get_expected_hidden_size(enc_inp, batch_sizes=[batch_size])
        dec_out, (hny, cny) = self.decoder(dec_emb,(hnx, cnx))

        return self.out_layer(dec_out)

# 以下確認作業
ds = psylex71_ds_s2p
s2p = Vec2Seq(
    sem_dim=ds.w2v.vector_size,
    dec_vocab_size=len(ds.phoneme),
    n_hid=n_hid,
    n_layers=n_layers,
    bidirectional=bidirectional).to(device)
print(s2p.eval())

## 3 Seq2Vec model

In [ ]:
class Seq2Vec(nn.Module):
    """ 系列データを符号化器に与え，埋め込みデータ (ベクトル) を復号化するモデル
    """
    def __init__(
        self,
        sem_dim:int,
        enc_vocab_size:int,
        n_hid:int,
        n_layers:int=2,
        bidirectional:bool=False):

        super().__init__()
        self.encoder_emb = nn.Embedding(
            num_embeddings=enc_vocab_size,
            embedding_dim=n_hid,
            padding_idx=0)

        self.encoder = nn.LSTM(
            input_size=n_hid,
            hidden_size=n_hid,
            num_layers=n_layers,
            batch_first=True,
            bidirectional=bidirectional)

        bi_fact = 2 if bidirectional else 1

        self.encoder_out = nn.Linear(
            in_features=n_hid * bi_fact,
            out_features=enc_vocab_size)

        self.out_layer = nn.Linear(
            in_features=n_hid * bi_fact,
            out_features=sem_dim)

    def forward(self, enc_inp):
        enc_emb = self.encoder_emb(enc_inp)
        enc_out, (hid, cel) = self.encoder(enc_emb)
        _enc_out = self.encoder_out(enc_out)
        _sem = self.out_layer(hid)
        return _sem, _enc_out

# 以下確認作業
ds = psylex71_ds_o2s
o2s = Seq2Vec(sem_dim=ds.w2v.vector_size,
              enc_vocab_size=len(ds.grapheme),
              n_layers=n_layers,
              bidirectional=bidirectional,
              n_hid=n_hid).to(device)
print(o2s.eval())

## 4 Vec2Vec model

In [ ]:
class Vec2Vec(nn.Module):
    """ ベクトル埋め込み表現をベクトル埋め込み表現へと変換
    """
    def __init__(self,
                 sem_dim:int,
                 n_hid:int):

        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(sem_dim, n_hid),
            nn.ReLU(),
            nn.Linear(n_hid, n_hid),
            nn.ReLU(),
            nn.Linear(n_hid, sem_dim),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# 以下確認作業
ds = psylex71_ds_s2s
s2s = Vec2Vec(sem_dim=ds.w2v.vector_size,
              n_hid=n_hid).to(device)
print(s2s.eval())

## 5 各モデルの宣言

In [ ]:
print(colored('# 1 写字モデル o2o', color='blue', attrs=["bold"]))
ds = psylex71_ds_o2o
o2o = Seq2Seq_wAtt(
    enc_vocab_size=len(ds.grapheme),
    dec_vocab_size=len(ds.grapheme),
    n_layers=n_layers, bidirectional=bidirectional, n_hid=n_hid).to(device)
print(o2o.eval())

print(colored('# 2 o2p 音読モデル意味関与なし', color='blue', attrs=["bold"]))
ds = psylex71_ds_o2p
o2p = Seq2Seq_wAtt(
    enc_vocab_size=len(ds.grapheme),
    dec_vocab_size=len(ds.phoneme),
    n_layers=n_layers, bidirectional=bidirectional, n_hid=n_hid).to(device)
print(o2p.eval())

print(colored('# 3 o2s 印字理解モデル', color='blue', attrs=["bold"]))
ds = psylex71_ds_o2s
o2s = Seq2Vec(
    enc_vocab_size=len(ds.grapheme),
    sem_dim=ds.w2v.vector_size,
    n_layers=n_layers, bidirectional=bidirectional, n_hid=n_hid).to(device)
print(o2s.eval())

print(colored('# 4 p2o 聞き書き ディクテーションモデル', color='blue', attrs=["bold"]))
ds = psylex71_ds_p2o
p2o = Seq2Seq_wAtt(
    enc_vocab_size=len(ds.phoneme),
    dec_vocab_size=len(ds.grapheme),
    n_layers=n_layers, bidirectional=bidirectional, n_hid=n_hid).to(device)
print(p2o.eval())

print(colored('# 5 p2p 復唱モデル', color='blue', attrs=["bold"]))
ds = psylex71_ds_p2p
p2p = Seq2Seq_wAtt(
    enc_vocab_size=len(ds.phoneme),
    dec_vocab_size=len(ds.phoneme),
    n_layers=n_layers, bidirectional=bidirectional, n_hid=n_hid).to(device)
print(p2p.eval())

print(colored('# 6 p2s 聴理解モデル', color='blue', attrs=["bold"]))
ds = psylex71_ds_p2s
p2s = Seq2Vec(
    enc_vocab_size=len(ds.phoneme),
    sem_dim=ds.w2v.vector_size,
    n_hid=n_hid, n_layers=n_layers, bidirectional=bidirectional).to(device)
print(p2s.eval())

print(colored('# 7 s2o 書字モデル', color='blue', attrs=["bold"]))
ds = psylex71_ds_s2o
s2o = Vec2Seq(
    sem_dim=ds.w2v.vector_size,
    dec_vocab_size=len(ds.grapheme),
    n_hid=n_hid, n_layers=n_layers, bidirectional=bidirectional).to(device)
print(s2o.eval())

print(colored('# 8 s2p 発話モデル', color='blue', attrs=["bold"]))
ds = psylex71_ds_s2p
s2p = Vec2Seq(
    sem_dim=ds.w2v.vector_size,
    dec_vocab_size=len(ds.phoneme),
    n_layers=n_layers, bidirectional=bidirectional, n_hid=n_hid).to(device)
print(s2p.eval())

print(colored('# 9 s2s 意味理解モデル', color='blue', attrs=["bold"]))
ds = psylex71_ds_s2s
s2s = Vec2Vec(
    sem_dim=len(ds.grapheme),
    n_hid=n_hid).to(device)
print(s2s.eval())

# 2 訓練手続きの定義

## `fit_seq2seq()`

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
def fit_seq2seq(
    model:torch.nn.modules.module.Module=o2p,
    epochs:int=10,
    ds:Dataset=psylex71_ds_o2p,
    batch_size=batch_size,
    collate_fn=_collate_fn,
    #dataloader:torch.utils.data.dataloader.DataLoader=dl_o2p,
    optimizer:torch.optim=None,
    lr:float=0.001,
    criterion:torch.nn.modules.loss=nn.CrossEntropyLoss(ignore_index=-1),
    interval:int=None,
    isPrint:bool=False,
    losses:list=None,
    isDraw:bool=True,):
    """ Seq2seq の訓練に用いる関数"""

    start_time = time.time()   # 開始時刻の保存

    dataloader = DataLoader(
        dataset=ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        collate_fn=collate_fn)

    if losses == None:
        losses = []

    model.train()

    if optimizer == None:
        optimizer = optim.Adam(model.parameters(), lr=lr)

    if interval == None:
        interval = int(ds.__len__()/batch_size) >> 2

    for epoch in range(epochs):
        i = 0
        for _inp, _tch in dataloader:
            enc_inp = pad_sequence(_inp, batch_first=True).to(device)
            dec_inp = pad_sequence(_tch, batch_first=True).to(device)
            tch = pad_sequence(_tch, batch_first=True, padding_value=-1.0).to(device)
            out = model(enc_inp, dec_inp)
            loss = criterion(out[0], tch[0])
            for h in range(1,len(tch)):
                loss += criterion(out[h], tch[h])
            losses.append(loss.item()/len(_inp))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            i += 1
            if (i % interval) == 0:
                print(f'epoch:{epoch+1:2d}',
                      f'batch:{i:2d}',
                      f'loss:{loss.item()/batch_size:.5f}')

    end_time = time.time()
    total_time = end_time - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))

    if isDraw:
        plt.plot(losses)
        plt.title(f'epochs:{epochs}, batch_size:{batch_size}, n_hid:{n_hid}, n_layers:{n_layers}, time collapsed:{total_time_str}')
        plt.show()

    return {'Training time':total_time_str,
            'losses': losses,
            'optimizer': optimizer,
            'time': total_time
           }

#fit_seq2seq(model=o2p, dataloader=dl_o2p) # 音読モデル
fit_seq2seq(epochs=1, model=o2p, ds=psylex71_ds_o2p); # 音読モデル
fit_seq2seq(epochs=1, model=p2p, ds=psylex71_ds_p2p); # 復唱モデル
fit_seq2seq(epochs=1, model=p2o, ds=psylex71_ds_p2o); # ディクテーションモデル
fit_seq2seq(epochs=1, model=o2o, ds=psylex71_ds_o2o); # 写字モデル

## `eval_seq2seq()`

In [ ]:
def eval_seq2seq(
    model:torch.nn.modules.module.Module=o2p,
    ds:Dataset=psylex71_ds_o2p,
    isPrint:bool=False,
    errors:list=None):

    model.eval()
    if errors == None:
        errors=[]

    for N in tqdm(range(ds.__len__())):
        x, y = ds.__getitem__(N)
        enc_inp, dec_inp = x.unsqueeze(0).to(device), y.unsqueeze(0).to(device)
        grand_truth = y.detach().numpy()[1:-1]
        y_hat = model(enc_inp, dec_inp).to('cpu')
        y_hat = np.argmax(y_hat.squeeze(0).detach().numpy(), axis=1)[1:-1]

        if len(y_hat) == len(grand_truth):
            n_correct = np.array((y_hat == grand_truth).sum())
            isOK = n_correct == len(grand_truth)
        else:
            isOK = False

        if not isOK:
            wrd = ds.getitem(N)[0]
            _out = ds.target_ids2target(y_hat)
            errors.append((N, wrd, _out,y_hat))
            #errors.append((N,y_hat))
            if isPrint:
                color = 'grey' if isOK else 'red'
                wrd = ds.getitem(N)[0]
                print(colored(f'{N:05d}', color),
                      colored(wrd, color='grey'), # , attrs=["bold"]),
                      colored(y_hat,color,attrs=["bold"]),
                      colored(ds.target_ids2target(y_hat), color, attrs=["bold"]),
                      f'<-{ds.target_ids2target(grand_truth)}')

    cr = len(errors) / N
    return {'エラー':errors,
            '正解率': (1.-cr) * 100}

In [ ]:
fit_seq2seq(epochs=3, model=s2p, ds=psylex71_ds_s2p)
eval_seq2seq(model=s2p, ds=psylex71_ds_s2p)

fit_seq2seq(epochs=3, model=s2o, ds=psylex71_ds_s2o)
eval_seq2seq(model=s2o, ds= psylex71_ds_s2o)

In [ ]:
#fit_seq2seq(epochs=15, model=s2o, ds=psylex71_ds_s2o)
out = eval_seq2seq(model=s2o, ds= psylex71_ds_s2o, isPrint=True)

In [ ]:
out

## `fit_vec2sec()` の定義
実際には `fit_seq2seq()` を用いるため何もしない。

## `eval_vec2sec()` は `eval_seq2seq()` と同じ

## `fit_seq2vec()` の定義

In [ ]:
def fit_seq2vec(
    model:torch.nn.modules.module.Module=o2p,
    epochs:int=10,
    ds:Dataset=psylex71_ds_o2s,
    batch_size=batch_size,
    collate_fn=_collate_fn,
    #dataloader:torch.utils.data.dataloader.DataLoader=dl_o2s,
    optimizer:torch.optim=None,
    lr:float=0.001,
    criterion_dec:torch.nn.modules.loss=nn.MSELoss(),
    criterion_enc:torch.nn.modules.loss=nn.CrossEntropyLoss(ignore_index=-1),
    interval:int=None,
    isPrint:bool=False,
    losses:list=None,
    isDraw:bool=True,):
    """ Seq2vec の訓練に用いる関数"""

    start_time = time.time()   # 開始時刻の保存

    dataloader = DataLoader(
        dataset=ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        collate_fn=collate_fn)


    if losses == None:
        losses = []

    model.train()

    if optimizer == None:
        optimizer = optim.Adam(model.parameters(), lr=lr)

    criterion_dec = nn.MSELoss()
    criterion_enc = nn.CrossEntropyLoss(ignore_index=-1)
    #criterion_enc = nn.CrossEntropyLoss(ignore_index=0)

    if interval == None:
        interval = int(ds.__len__()/batch_size) >> 2

    for epoch in range(epochs):
        i = 0
        for _inp, _tch in dataloader:
            enc_inp = pad_sequence(_inp, batch_first=True).to(device)
            tch = pad_sequence(_tch, batch_first=True, padding_value=-1.0).to(device)
            out, enc_out = model(enc_inp)

            out = out.squeeze(0)
            loss = criterion_dec(out, tch)
            for _x, _y in zip(enc_out[:,:-1,:], enc_inp[:,1:]):
                loss += criterion_enc(_x, _y)
            losses.append(loss.item()/len(_inp))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            i += 1
            if (i % interval) == 0:
                print(f'epoch:{epoch+1:2d}',
                      f'batch:{i:2d}',
                      f'loss:{loss.item()/batch_size:.5f}')

    end_time = time.time()
    total_time = end_time - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))

    if isDraw:
        plt.plot(losses)
        plt.title(f'epochs:{epochs}, batch_size:{batch_size}, n_hid:{n_hid}, n_layers:{n_layers}, time collapsed:{total_time_str}')
        plt.show()

    return {'Training time': total_time_str,
            'losses': losses,
            'optimizer': optimizer,
            'time': total_time }

#fit_seq2vec(epochs=10, model=p2s, lr=0.0001, ds=psylex71_ds_p2s, dataloader=dl_p2s); # 聴理解モデル
#fit_seq2vec(epochs=50, model=o2s, lr=0.0001, ds=psylex71_ds_o2s, dataloader=dl_o2s); # 印字理解モデル
#fit_seq2vec(epochs=20, model=o2s, lr=0.00001, ds=psylex71_ds_o2s, dataloader=dl_o2s); # 印字理解モデル
fit_seq2vec(epochs=20, model=o2s, lr=0.0001, ds=psylex71_ds_o2s); # 印字理解モデル

In [ ]:
start_time = time.time()
n_hid = 64
#n_hid = 32
n_layers = 1
epochs = 10
bidirectional=False
model = EncDec_s2p(sem_dim=_psylex71_ds.w2v.vector_size,
                   dec_vocab_size=len(_psylex71_ds.phoneme),
                   n_layers=n_layers,
                   bidirectional=bidirectional,
                   n_hid=n_hid).to(device)

optimizer = optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=-1)

model.train()
interval = int(_psylex71_ds.__len__()/batch_size) >> 2
losses = []
for epoch in range(epochs):
    i = 0
    for x, y in dataloader:
        print(type(x), len(x))
        sys.exit()
        enc_inp = torch.from_numpy(np.array(x)).float().to(device).unsqueeze(0)
        #enc_inp = torch.from_numpy(np.array(x)).to(device).unsqueeze(0)
        dec_inp = pad_sequence(y, batch_first=True).to(device)[:,1:]
        tch = pad_sequence(y, batch_first=True, padding_value=-1.0).to(device)
        tch = tch[:,1:]
        out = model(enc_inp, dec_inp)
        loss = criterion(out[0], tch[0])
        for h in range(1,len(tch)):
            loss += criterion(out[h], tch[h])
        losses.append(loss.item()/len(x))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        if (i % interval) == 0:
            print(f'epoch:{epoch:2d}, batch:{i:2d}, loss:{loss.item()/len(x):.3f}')

end_time = time.time()
total_time = end_time - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print(f'Training time {total_time_str}')

plt.plot(losses)
plt.title(f'epochs:{epochs}, batch_size:{batch_size}, n_hid:{n_hid}, n_layers:{n_layers}, time collapsed:{total_time_str}')
plt.show()

#outfile = "attnmt2-" + str(epoch) + ".model"
#torch.save(net.state_dict(),outfile)

## 2.2 学習結果の評価

In [ ]:
errors = []
model.eval()
for N in range(_psylex71_ds.__len__()):
#for N in np.random.permutation(_psylex71_ds.__len__())[:100]:
    x, y = _psylex71_ds.__getitem__(N)
    enc_inp = torch.from_numpy(np.array(x)).to(device).unsqueeze(0)
    enc_emb = model.enc_transform_layer(enc_inp)
    hnx, cnx = enc_emb.clone(), enc_emb.clone()
    dec_inp = y
    dec_emb = model.decoder_emb(dec_inp)
    dec_out, (hny, cny) = model.decoder(dec_emb,(hnx, cnx))
    dec_out = model.out_layer(dec_out)
    y_ids = np.argmax(dec_out.detach().numpy(),axis=1)

    n_correct = np.array((y_ids[1:-1] == _psylex71_ds.getitem(N)[2]).sum())
    isOK = n_correct == len(_psylex71_ds.getitem(N)[2])
    color = 'grey' if isOK else 'red'

    if not isOK:
        errors.append((N,y_ids))
        print(colored((f'{N:05d}', #y_ids,
                       "".join(p for p in _psylex71_ds.phon_ids2phn(y_ids[1:-1]))),color,attrs=["bold"]), end=" ")
        print(_psylex71_ds.getitem(N))

cr = len(errors) / _psylex71_ds.__len__()
print(f'総エラー数:{len(errors)}',
      f'正解率:{(1.-cr)*100:.3f}')
